In [1]:
import pandas as pd
import haversine as hs
import warnings
from datetime import datetime
import numpy as np
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)

df  = pd.read_csv('test_task.csv')

### Комментарий к выполнению задания

1. Реализован пайплайн для обработки одного конкретного объекта.
2. Обработка большого числа объектов предлагается следующим образом: 
- Складываем пачки объектов приемлемого размера для pandas в отдельные файлики
- используя multiprocessing.Pool обрабатываем файлы, разбирая внутри на отдельные объекты и применяя пайплайн для обработки одного объекта 
- сохраняем выходные датафреймы пачками приемлемого размера
- *Блок не реализован в коде*
3. Визуализацию на карте можно попробовать с gmplot. Если же захотим строить только трек перемещения, то можно попробовать с matplotlib для простоты
- *Блок не реализован*


#### Алгоритм расчета основных параметров поездки: 
1. Выбираем конкретный объект, сортируем в порядке возрастания по времени событий. Сортировку можно выбросить для ускорения процесса если мы точно знаем, что все записи будут уже отсортированными <br><br>
2. Для каждой записи подтягиваем предыдущее время и координаты. Считаем дельту по времени в секундах и расстояние между координатами в метрах. <br><br>
3. Считаем скорость как отношение расстояния к дельте по времени. Заменяем на ноль в случае если дельта времени равна нулю.<br><br>
4. Считаем ускорение как абсолютное значение отношения дельты скорости к дельте по времени. Заменяем на ноль в случае если дельта времени равна нулю.<br><br>
5. Считаем взвешенные скорости и ускорения: для каждой записи скорость и ускорение умножаем на дельту времени.<br><br>
6. Формируем признак движения - если пройденное расстояние больше нуля, то 1. Иначе 0.<br><br>
7. Проходим в цикле по каждой записи и размечаем поездки: если текущая и предыдущая записи имеют совпадающий признак движения, то размечаем одним и тем же trip_id (для первой поездки trip_id будет равен нулю). Если же признак движения различается у текущей и предыдущей записей, то для текущей записи trip_id = trip_id + 1. <br><br>
8. Группируем по trip_id, считаем длительность поездки, пройденное расстояние, средние скорости и ускорения (отношение суммы взвешенных значений к общей длительности поездки)<br><br>
9. Получаем аутпут-датафрейм.

In [184]:
r = 8

In [189]:
i = 0
j = 7

In [199]:
(df[df['id'] == 1]) is True

False

In [204]:
df[df['id'] == 900].empty

True

In [210]:
j = 0 

while not df.loc[df['id'] == j].empty:
    j += 8
    print([df.loc[df['id'] == x] for x in range(j, j + 8) if not df.loc[df['id'] == x].empty])

[             lat        lon                    t  id
40000  58.586137  49.615083  2022-03-07 12:55:21   8
40001  58.586137  49.615083  2022-03-07 13:05:20   8
40002  58.586137  49.615083  2022-03-07 13:15:20   8
40003  58.586137  49.615083  2022-03-07 13:25:20   8
40004  58.586137  49.615083  2022-03-07 13:35:20   8
...          ...        ...                  ...  ..
44995  58.623212  49.662872  2022-03-18 07:02:09   8
44996  58.623212  49.662872  2022-03-18 07:12:09   8
44997  58.623212  49.662872  2022-03-18 07:22:09   8
44998  58.623212  49.662875  2022-03-18 07:32:09   8
44999  58.623212  49.662875  2022-03-18 07:42:09   8

[5000 rows x 4 columns],              lat        lon                    t  id
45000  58.593627  49.649913  2022-03-10 13:35:19   9
45001  58.593623  49.649743  2022-03-10 13:35:21   9
45002  58.593553  49.647480  2022-03-10 13:35:36   9
45003  58.593518  49.647217  2022-03-10 13:35:40   9
45004  58.593533  49.647000  2022-03-10 13:35:42   9
...          ...   

[              lat        lon                    t  id
155088  28.567281  77.133985  2022-03-11 08:17:41  32
155089  28.568371  77.134050  2022-03-11 08:17:52  32
155090  28.569475  77.134160  2022-03-11 08:18:00  32
155091  28.570440  77.134508  2022-03-11 08:18:12  32
155092  28.571293  77.135171  2022-03-11 08:18:20  32
...           ...        ...                  ...  ..
160083  28.169895  76.691065  2022-03-12 10:59:27  32
160084  28.169895  76.691065  2022-03-12 10:59:37  32
160085  28.169895  76.691065  2022-03-12 10:59:47  32
160086  28.169895  76.691065  2022-03-12 10:59:57  32
160087  28.169895  76.691065  2022-03-12 10:59:57  32

[5000 rows x 4 columns],              lat        lon                    t  id
160088 -27.41333  152.97402  2022-03-09 20:20:39  33
160089 -27.41326  152.97395  2022-03-09 20:20:42  33
160090 -27.41197  152.97131  2022-03-09 20:21:22  33
160091 -27.41034  152.96799  2022-03-09 20:22:22  33
160092 -27.40998  152.96689  2022-03-09 20:22:31  33
...    

[              lat        lon                    t  id
235086  54.566753  36.351162  2022-03-14 16:24:01  48
235087  54.566173  36.350892  2022-03-14 16:24:11  48
235088  54.565560  36.350770  2022-03-14 16:24:23  48
235089  54.565452  36.351148  2022-03-14 16:24:27  48
235090  54.565425  36.351387  2022-03-14 16:24:28  48
...           ...        ...                  ...  ..
240081  46.309478  44.233198  2022-03-18 07:33:45  48
240082  46.309478  44.233198  2022-03-18 07:37:45  48
240083  46.309478  44.233198  2022-03-18 07:41:45  48
240084  46.309478  44.233198  2022-03-18 07:45:45  48
240085  46.309478  44.233198  2022-03-18 07:49:45  48

[5000 rows x 4 columns],              lat        lon                    t  id
240086  6.281777 -75.644924  2022-03-15 01:45:26  49
240087  6.281777 -75.644924  2022-03-15 01:45:26  49
240088  6.281777 -75.644924  2022-03-15 01:45:39  49
240089  6.281803 -75.644924  2022-03-15 01:45:52  49
240090  6.281790 -75.644898  2022-03-15 01:47:22  49
...    

[              lat        lon                    t  id
314960  47.086307  51.930202  2022-03-03 05:07:13  64
314961  47.086395  51.930172  2022-03-03 05:07:37  64
314962  47.087002  51.931267  2022-03-03 05:08:02  64
314963  47.086692  51.931613  2022-03-03 05:08:49  64
314964  47.086692  51.931613  2022-03-03 05:08:57  64
...           ...        ...                  ...  ..
319955  43.429607  52.365665  2022-03-18 07:51:08  64
319956  43.431562  52.359868  2022-03-18 07:51:34  64
319957  43.433130  52.354045  2022-03-18 07:51:59  64
319958  43.434892  52.348165  2022-03-18 07:52:25  64
319959  43.436658  52.342395  2022-03-18 07:52:51  64

[5000 rows x 4 columns],               lat        lon                    t  id
319960 -12.049286 -76.961113  2022-03-16 23:38:00  65
319961 -12.049286 -76.961113  2022-03-16 23:38:24  65
319962 -12.049286 -76.961113  2022-03-16 23:38:54  65
319963 -12.049286 -76.961113  2022-03-16 23:39:24  65
319964 -12.049286 -76.961113  2022-03-16 23:39:56  65
.

[              lat        lon                    t  id
474960 -29.713667 -57.215765  2022-03-08 02:43:13  96
474961 -29.713667 -57.215765  2022-03-08 02:44:56  96
474962 -29.713667 -57.215765  2022-03-08 02:45:09  96
474963 -29.713667 -57.215765  2022-03-08 02:45:36  96
474964 -29.713667 -57.215765  2022-03-08 02:45:56  96
...           ...        ...                  ...  ..
479955 -25.458105 -50.620773  2022-03-18 05:43:29  96
479956 -25.458105 -50.620773  2022-03-18 06:13:29  96
479957 -25.458105 -50.620773  2022-03-18 06:43:29  96
479958 -25.458105 -50.620773  2022-03-18 07:13:29  96
479959 -25.458105 -50.620773  2022-03-18 07:43:29  96

[5000 rows x 4 columns]]
[]


In [213]:
multiprocessing.cpu_count()

8

In [212]:
import multiprocessing

In [193]:
[df.loc[df['id'] == x] for x in range(i, i + 8)]

[           lat         lon                    t  id
 0    -6.257188  106.946373  2022-03-15 12:24:58   0
 1    -6.257284  106.944613  2022-03-15 12:25:08   0
 2    -6.257325  106.943147  2022-03-15 12:25:18   0
 3    -6.257380  106.942151  2022-03-15 12:25:30   0
 4    -6.257436  106.941049  2022-03-15 12:25:38   0
 ...        ...         ...                  ...  ..
 4995 -6.881787  109.410693  2022-03-18 07:05:09   0
 4996 -6.881926  109.411147  2022-03-18 07:05:19   0
 4997 -6.881983  109.411458  2022-03-18 07:05:29   0
 4998 -6.881997  109.411609  2022-03-18 07:05:49   0
 4999 -6.881997  109.411609  2022-03-18 07:06:55   0
 
 [5000 rows x 4 columns],
            lat         lon                    t  id
 5000 -6.928387  109.762711  2022-03-15 08:01:54   1
 5001 -6.928170  109.763529  2022-03-15 08:02:04   1
 5002 -6.927958  109.764338  2022-03-15 08:04:30   1
 5003 -6.927736  109.765218  2022-03-15 08:04:30   1
 5004 -6.927493  109.766142  2022-03-15 08:04:30   1
 ...        ...   

In [187]:
for i in range(i, i + 8):
    print([x for x in i])

0
1
2
3
4
5
6
7


In [183]:
df.loc[df['id'] == 1]

,lat,lon,t,id
5000,-6.928387,109.762711,2022-03-15 08:01:54,1
5001,-6.928170,109.763529,2022-03-15 08:02:04,1
5002,-6.927958,109.764338,2022-03-15 08:04:30,1
5003,-6.927736,109.765218,2022-03-15 08:04:30,1
5004,-6.927493,109.766142,2022-03-15 08:04:30,1
...,...,...,...,...
9995,-7.708174,112.863636,2022-03-16 23:42:57,1
9996,-7.708118,112.863591,2022-03-17 02:56:27,1
9997,-7.708279,112.863627,2022-03-17 03:05:23,1
9998,-7.708279,112.863627,2022-03-17 04:38:25,1


In [2]:
import pandas as pd
import haversine as hs
import warnings
from datetime import datetime
import numpy as np
warnings.filterwarnings('ignore')

# Функция для подготовки поездок конкретного объекта
def data_transforming(tdf):    
    # Сортировка необходима, если мы не знаем, что файл гаранитрованно отсортирован
    # В нашем случае файл уже отсортирован, но оставим сортировку для общего случая
    tdf['t'] = tdf['t'].astype('datetime64')
    tdf.sort_values(by = 't', inplace = True)
    
    # Дельта времени между записями
    tdf['t_diff'] = tdf['t'].diff(1).dt.seconds

    # Вычисление перемещений 
    tdf['lat_prev'] = tdf['lat'].shift(1)
    tdf['lon_prev'] = tdf['lon'].shift(1)
    tdf['coord'] = tdf.apply(lambda x: tuple(x[['lat','lon']]),axis = 1)
    tdf['coord_prev'] = tdf.apply(lambda x: tuple(x[['lat_prev','lon_prev']]),axis = 1)
    tdf['dist'] = tdf[['coord', 'coord_prev']].apply(lambda x: hs.haversine(x['coord'], x['coord_prev']), axis = 1).fillna(0) * 1000
    
    # Вычисление кусочных скоростей и ускорений 
    tdf['velocity'] = (tdf['dist']/tdf['t_diff']).apply(lambda x: 0 if  np.isinf(x) else x).fillna(0)
    tdf['acceleration'] = np.abs((tdf['velocity'].diff(1)/tdf['t_diff']).apply(lambda x: 0 if  np.isinf(x) else x)).fillna(0)

    # Флаг признака движения - поможет сгруппировать кусочки поездки в общую поездку
    tdf['movement_flg'] = tdf['dist'].apply(lambda x: 0 if x == 0 else 1)
    
    # Взвешенные скорости и ускорения. Необходимы для вычисления средних ускорений и скоростей.
    tdf['velocity_w'] = tdf['velocity'] * tdf['t_diff']
    tdf['acceleration_w'] = tdf['acceleration'] * tdf['t_diff']
    print(tdf['id'].unique())
    # Пробег в цикле и разметка уникальных поездок. 
    # Необходима для дальнейшей агрегации и вычисления расстояний, скоростей и ускорений 
    j = 0 
    for i in tdf.index[1:]:
        if tdf.loc[i, 'movement_flg'] == tdf.loc[i - 1, 'movement_flg']:
            tdf.loc[i, 'trip_id'] = j
        else:
            j += 1
            tdf.loc[i, 'trip_id'] = j
    
    # Формирование финального результата 
    
    output = pd.concat([tdf.groupby(['trip_id'])['dist'].sum(),
                        tdf.groupby(['trip_id'])['t_diff'].sum().rename('trip_time'),
                       (tdf.groupby('trip_id')['velocity_w'].sum()/ tdf.groupby('trip_id')['t_diff'].sum()).rename('velocity_mean'),
                       (tdf.groupby('trip_id')['acceleration_w'].sum()/ tdf.groupby('trip_id')['t_diff'].sum()).rename('acceleration_mean')], axis = 1)

    # Очистка от простоев и кратких поездок
    output = output.loc[(output['dist'] > 0 ) & (output['trip_time'] >= 60)]
    return output

In [50]:
tdf = df[df['id'] == 0].copy()

In [51]:
# Сортировка необходима, если мы не знаем, что файл гаранитрованно отсортирован
# В нашем случае файл уже отсортирован, но оставим сортировку для общего случая
tdf['t'] = tdf['t'].astype('datetime64')
#tdf = tdf.sort_values('t').drop_duplicates('t',keep='last').reset_index(drop = True)
tdf.sort_values(by = 't', inplace = True)

# Дельта времени между записями
tdf['t_diff'] = tdf['t'].diff(1).dt.seconds

# Вычисление перемещений 
tdf['lat_prev'] = tdf['lat'].shift(1)
tdf['lon_prev'] = tdf['lon'].shift(1)
tdf['coord'] = tdf.apply(lambda x: tuple(x[['lat','lon']]),axis = 1)
tdf['coord_prev'] = tdf.apply(lambda x: tuple(x[['lat_prev','lon_prev']]),axis = 1)
tdf['dist'] = tdf[['coord', 'coord_prev']].apply(lambda x: hs.haversine(x['coord'], x['coord_prev']), axis = 1).fillna(0) * 1000

# Вычисление кусочных скоростей и ускорений 
tdf['velocity'] = (tdf['dist']/tdf['t_diff']).apply(lambda x: 0 if  np.isinf(x) else x).fillna(0)
tdf['acceleration'] = np.abs((tdf['velocity'].diff(1)/tdf['t_diff']).apply(lambda x: 0 if  np.isinf(x) else x)).fillna(0)

# Флаг признака движения - поможет сгруппировать кусочки поездки в общую поездку
tdf['movement_flg'] = tdf['dist'].apply(lambda x: 0 if x == 0 else 1)

# Взвешенные скорости и ускорения. Необходимы для вычисления средних ускорений и скоростей.
tdf['velocity_w'] = tdf['velocity'] * tdf['t_diff']
tdf['acceleration_w'] = tdf['acceleration'] * tdf['t_diff']
print(tdf['id'].unique())
# Пробег в цикле и разметка уникальных поездок. 
# Необходима для дальнейшей агрегации и вычисления расстояний, скоростей и ускорений 
j = 0 
for i in tdf.index[1:]:
    if tdf.loc[i, 'movement_flg'] == tdf.loc[i - 1, 'movement_flg']:
        tdf.loc[i, 'trip_id'] = j
    else:
        j += 1
        tdf.loc[i, 'trip_id'] = j

[0]


In [45]:
tdf = df[df['id'] == 0].copy()

In [32]:
pd.set_option('use_inf_as_na', True)

In [176]:
# Векторизированная функция расчета расстояния между координатами
def haversine_dist(s_lat, s_lng, e_lat, e_lng):
    R = 6371
    s_lat = s_lat*np.pi/180.0                      
    s_lng = np.deg2rad(s_lng)     
    e_lat = np.deg2rad(e_lat)                       
    e_lng = np.deg2rad(e_lng)  
    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2
    return 2 * R * np.arcsin(np.sqrt(d))

# Функция вычисления расстояния для поездок 
def distance_calculation(tdf):
    # Сортировка и приведение к формату даты 
    tdf['t'] = tdf['t'].astype('datetime64')
    tdf.sort_values(by = 't', inplace = True)
    
    # Дельта времени между записями
    tdf['t_diff'] = tdf['t'].diff(1).dt.seconds

    # Вычисление перемещения
    tdf['dist'] = haversine_dist(tdf['lat'].shift(1), tdf['lon'].shift(1), tdf['lat'], tdf['lon']).fillna(0) * 1000
    
# Функция вычисления кусочной скорости
def velocity_calculation(tdf):
    tdf['velocity'] = (tdf['dist']/tdf['t_diff']).fillna(0)
    
    
# Функция вычисления кусочного ускорения
def acceleration_calculation(tdf):
    tdf['acceleration'] = np.abs(tdf['velocity'].diff(1)/tdf['t_diff']).fillna(0)
    
    
# Схлопывание результата на уровень поездок
def aggregation(tdf, agg_level = 'acceleration'):
    j = 0 
    for i in tdf.index[1:]:
        if (tdf.loc[i, 'dist'] > 0) == (tdf.loc[i - 1, 'dist'] > 0):
            tdf.loc[i, 'trip_id'] = j
        else:
            j += 1
            tdf.loc[i, 'trip_id'] = j
    
    # выбрасываем нерелевантные кусочки
    tdf = tdf.loc[tdf['t_diff'] != 0]
    # Формирование финального результата 
    if agg_level in ['velocity', 'acceleration', 'distance']:
        
        output = pd.concat([tdf.groupby(['trip_id'])['id'].min(),
                            tdf.groupby(['trip_id'])['dist'].sum(),
                                tdf.groupby(['trip_id'])['t_diff'].sum().rename('trip_time')], axis = 1)
    
    if agg_level in ['velocity', 'acceleration']:
        
        tdf['velocity_w'] = tdf['velocity'] * tdf['t_diff']
        output = pd.concat([output,
                           (tdf.groupby('trip_id')['velocity_w'].sum()/ tdf.groupby('trip_id')['t_diff'].sum()).rename('velocity_mean')], axis = 1)
    
    if agg_level == 'acceleration':
        
        tdf['acceleration_w'] = tdf['acceleration'] * tdf['t_diff']
        output = pd.concat([output,
                           (tdf.groupby('trip_id')['acceleration_w'].sum()/ tdf.groupby('trip_id')['t_diff'].sum()).rename('acceleration_mean')], axis = 1)
        
    # Очистка от простоев и кратких поездок
    return output.loc[(output['dist'] > 0 ) & (output['trip_time'] >= 60)]



def calculation_funct(tdf, agg_level = 'velocity'):
    if agg_level in ['velocity', 'acceleration', 'distance']:
        distance_calculation(sdf)
        
    if agg_level in ['velocity', 'acceleration']:
        velocity_calculation(sdf)
        
    if agg_level in ['acceleration']:
        acceleration_calculation(sdf)
    
    return aggregation(sdf, agg_level = agg_level)

In [181]:
calculation_funct(sdf, agg_level = 'acceleration')

,id,dist,trip_time,velocity_mean,acceleration_mean
trip_id,,,,,
1.0,0,76700.161545,67121.0,1.142715,0.067403
3.0,0,59828.108519,16662.0,3.590692,0.211866
5.0,0,1873.274924,27693.0,0.067644,0.005801
7.0,0,441.937960,1720.0,0.256941,0.015902
9.0,0,13.910218,273.0,0.050953,0.000187
11.0,0,320838.497065,76283.0,4.205898,0.368722
13.0,0,3134.595756,549.0,5.709646,0.215554


In [158]:
sdf

,lat,lon,t,id,t_diff,dist,velocity,trip_id
0,-6.257188,106.946373,2022-03-15 12:24:58,0,NaN,0.000000,0.000000,NaN
1,-6.257284,106.944613,2022-03-15 12:25:08,0,10.0,194.830516,19.483052,1.0
2,-6.257325,106.943147,2022-03-15 12:25:18,0,10.0,162.178740,16.217874,1.0
3,-6.257380,106.942151,2022-03-15 12:25:30,0,12.0,110.211020,9.184252,1.0
4,-6.257436,106.941049,2022-03-15 12:25:38,0,8.0,121.987856,15.248482,1.0
...,...,...,...,...,...,...,...,...
4995,-6.881787,109.410693,2022-03-18 07:05:09,0,10.0,63.422043,6.342204,13.0
4996,-6.881926,109.411147,2022-03-18 07:05:19,0,10.0,52.355764,5.235576,13.0
4997,-6.881983,109.411458,2022-03-18 07:05:29,0,10.0,34.940470,3.494047,13.0
4998,-6.881997,109.411609,2022-03-18 07:05:49,0,20.0,16.747467,0.837373,13.0


In [180]:
sdf = df[df['id'] == 0].copy()

In [142]:
distance_calculation(sdf)

In [143]:
velocity_calculation(sdf)

In [144]:
acceleration_calculation(sdf)

In [148]:
aggregation(sdf, agg_level = 's')

KeyError: 'dist'

In [122]:
sdf

,lat,lon,t,id,t_diff,dist,velocity,acceleration
0,-6.257188,106.946373,2022-03-15 12:24:58,0,NaN,0.000000,0.000000,0.000000
1,-6.257284,106.944613,2022-03-15 12:25:08,0,10.0,194.830516,19.483052,1.948305
2,-6.257325,106.943147,2022-03-15 12:25:18,0,10.0,162.178740,16.217874,0.326518
3,-6.257380,106.942151,2022-03-15 12:25:30,0,12.0,110.211020,9.184252,0.586135
4,-6.257436,106.941049,2022-03-15 12:25:38,0,8.0,121.987856,15.248482,0.758029
...,...,...,...,...,...,...,...,...
4995,-6.881787,109.410693,2022-03-18 07:05:09,0,10.0,63.422043,6.342204,0.051773
4996,-6.881926,109.411147,2022-03-18 07:05:19,0,10.0,52.355764,5.235576,0.110663
4997,-6.881983,109.411458,2022-03-18 07:05:29,0,10.0,34.940470,3.494047,0.174153
4998,-6.881997,109.411609,2022-03-18 07:05:49,0,20.0,16.747467,0.837373,0.132834


In [98]:
sdf

,lat,lon,t,id
0,-6.257188,106.946373,2022-03-15 12:24:58,0
1,-6.257284,106.944613,2022-03-15 12:25:08,0
2,-6.257325,106.943147,2022-03-15 12:25:18,0
3,-6.257380,106.942151,2022-03-15 12:25:30,0
4,-6.257436,106.941049,2022-03-15 12:25:38,0
...,...,...,...,...
4995,-6.881787,109.410693,2022-03-18 07:05:09,0
4996,-6.881926,109.411147,2022-03-18 07:05:19,0
4997,-6.881983,109.411458,2022-03-18 07:05:29,0
4998,-6.881997,109.411609,2022-03-18 07:05:49,0


In [99]:
distance_calculation(sdf)

In [100]:
sdf

,lat,lon,t,id,t_diff,dist
0,-6.257188,106.946373,2022-03-15 12:24:58,0,NaN,0.000000
1,-6.257284,106.944613,2022-03-15 12:25:08,0,10.0,194.830516
2,-6.257325,106.943147,2022-03-15 12:25:18,0,10.0,162.178740
3,-6.257380,106.942151,2022-03-15 12:25:30,0,12.0,110.211020
4,-6.257436,106.941049,2022-03-15 12:25:38,0,8.0,121.987856
...,...,...,...,...,...,...
4995,-6.881787,109.410693,2022-03-18 07:05:09,0,10.0,63.422043
4996,-6.881926,109.411147,2022-03-18 07:05:19,0,10.0,52.355764
4997,-6.881983,109.411458,2022-03-18 07:05:29,0,10.0,34.940470
4998,-6.881997,109.411609,2022-03-18 07:05:49,0,20.0,16.747467


In [101]:
velocity_calculation(sdf)

In [103]:
acceleration_calculation(sdf)

In [104]:
sdf

,lat,lon,t,id,t_diff,dist,velocity,acceleration
0,-6.257188,106.946373,2022-03-15 12:24:58,0,NaN,0.000000,NaN,NaN
1,-6.257284,106.944613,2022-03-15 12:25:08,0,10.0,194.830516,19.483052,NaN
2,-6.257325,106.943147,2022-03-15 12:25:18,0,10.0,162.178740,16.217874,0.326518
3,-6.257380,106.942151,2022-03-15 12:25:30,0,12.0,110.211020,9.184252,0.586135
4,-6.257436,106.941049,2022-03-15 12:25:38,0,8.0,121.987856,15.248482,0.758029
...,...,...,...,...,...,...,...,...
4995,-6.881787,109.410693,2022-03-18 07:05:09,0,10.0,63.422043,6.342204,0.051773
4996,-6.881926,109.411147,2022-03-18 07:05:19,0,10.0,52.355764,5.235576,0.110663
4997,-6.881983,109.411458,2022-03-18 07:05:29,0,10.0,34.940470,3.494047,0.174153
4998,-6.881997,109.411609,2022-03-18 07:05:49,0,20.0,16.747467,0.837373,0.132834


In [102]:
sdf

,lat,lon,t,id,t_diff,dist,velocity
0,-6.257188,106.946373,2022-03-15 12:24:58,0,NaN,0.000000,NaN
1,-6.257284,106.944613,2022-03-15 12:25:08,0,10.0,194.830516,19.483052
2,-6.257325,106.943147,2022-03-15 12:25:18,0,10.0,162.178740,16.217874
3,-6.257380,106.942151,2022-03-15 12:25:30,0,12.0,110.211020,9.184252
4,-6.257436,106.941049,2022-03-15 12:25:38,0,8.0,121.987856,15.248482
...,...,...,...,...,...,...,...
4995,-6.881787,109.410693,2022-03-18 07:05:09,0,10.0,63.422043,6.342204
4996,-6.881926,109.411147,2022-03-18 07:05:19,0,10.0,52.355764,5.235576
4997,-6.881983,109.411458,2022-03-18 07:05:29,0,10.0,34.940470,3.494047
4998,-6.881997,109.411609,2022-03-18 07:05:49,0,20.0,16.747467,0.837373


In [93]:
(5 > 0) == (-1 > 0)

False

In [ ]:
# Флаг признака движения - поможет сгруппировать кусочки поездки в общую поездку
tdf['movement_flg'] = tdf['dist'].apply(lambda x: 0 if x == 0 else 1)

# Взвешенные скорости и ускорения. Необходимы для вычисления средних ускорений и скоростей.
tdf['velocity_w'] = (tdf['velocity'] * tdf['t_diff']).fillna(0)
tdf['acceleration_w'] = (tdf['acceleration'] * tdf['t_diff']).fillna(0)
print(tdf['id'].unique())
# Пробег в цикле и разметка уникальных поездок. 
# Необходима для дальнейшей агрегации и вычисления расстояний, скоростей и ускорений 
j = 0 
for i in tdf.index[1:]:
    if tdf.loc[i, 'movement_flg'] == tdf.loc[i - 1, 'movement_flg']:
        tdf.loc[i, 'trip_id'] = j
    else:
        j += 1
        tdf.loc[i, 'trip_id'] = j

tdf = tdf.loc[tdf['t_diff'] != 0]
# Формирование финального результата 

output = pd.concat([tdf.groupby(['trip_id'])['dist'].sum(),
                    tdf.groupby(['trip_id'])['t_diff'].sum().rename('trip_time'),
                   (tdf.groupby('trip_id')['velocity_w'].sum()/ tdf.groupby('trip_id')['t_diff'].sum()).rename('velocity_mean'),
                   (tdf.groupby('trip_id')['acceleration_w'].sum()/ tdf.groupby('trip_id')['t_diff'].sum()).rename('acceleration_mean')], axis = 1)

# Очистка от простоев и кратких поездок
output = output.loc[(output['dist'] > 0 ) & (output['trip_time'] >= 60)]

In [64]:
import pandas as pd
import warnings
from datetime import datetime
import numpy as np
warnings.filterwarnings('ignore')

tdf = df[df['id'] == 0].copy()

# Векторизированная функция расчета расстояния между координатами
def distance(s_lat, s_lng, e_lat, e_lng):
    R = 6371
    s_lat = s_lat*np.pi/180.0                      
    s_lng = np.deg2rad(s_lng)     
    e_lat = np.deg2rad(e_lat)                       
    e_lng = np.deg2rad(e_lng)  
    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2
    return 2 * R * np.arcsin(np.sqrt(d))


# Функция для подготовки поездок конкретного объекта

# Сортировка необходима, если мы не знаем, что файл гаранитрованно отсортирован
# В нашем случае файл уже отсортирован, но оставим сортировку для общего случая
tdf['t'] = tdf['t'].astype('datetime64')
tdf.sort_values(by = 't', inplace = True)

# Дельта времени между записями
tdf['t_diff'] = tdf['t'].diff(1).dt.seconds

# Вычисление перемещения
tdf['dist'] = distance(tdf['lat'].shift(1), tdf['lon'].shift(1), tdf['lat'], tdf['lon']).fillna(0) * 1000

# Вычисление кусочных скоростей и ускорений 
tdf['velocity'] = tdf['dist']/tdf['t_diff']
tdf['acceleration'] = np.abs(tdf['velocity'].diff(1)/tdf['t_diff'])

#tdf['velocity'] = (tdf['dist']/tdf['t_diff']).apply(lambda x: 0 if  np.isinf(x) else x).fillna(0)
#tdf['acceleration'] = np.abs((tdf['velocity'].diff(1)/tdf['t_diff']).apply(lambda x: 0 if  np.isinf(x) else x)).fillna(0)

# Флаг признака движения - поможет сгруппировать кусочки поездки в общую поездку
tdf['movement_flg'] = tdf['dist'].apply(lambda x: 0 if x == 0 else 1)

# Взвешенные скорости и ускорения. Необходимы для вычисления средних ускорений и скоростей.
tdf['velocity_w'] = (tdf['velocity'] * tdf['t_diff']).fillna(0)
tdf['acceleration_w'] = (tdf['acceleration'] * tdf['t_diff']).fillna(0)
print(tdf['id'].unique())
# Пробег в цикле и разметка уникальных поездок. 
# Необходима для дальнейшей агрегации и вычисления расстояний, скоростей и ускорений 
j = 0 
for i in tdf.index[1:]:
    if tdf.loc[i, 'movement_flg'] == tdf.loc[i - 1, 'movement_flg']:
        tdf.loc[i, 'trip_id'] = j
    else:
        j += 1
        tdf.loc[i, 'trip_id'] = j

tdf = tdf.loc[tdf['t_diff'] != 0]
# Формирование финального результата 

output = pd.concat([tdf.groupby(['trip_id'])['dist'].sum(),
                    tdf.groupby(['trip_id'])['t_diff'].sum().rename('trip_time'),
                   (tdf.groupby('trip_id')['velocity_w'].sum()/ tdf.groupby('trip_id')['t_diff'].sum()).rename('velocity_mean'),
                   (tdf.groupby('trip_id')['acceleration_w'].sum()/ tdf.groupby('trip_id')['t_diff'].sum()).rename('acceleration_mean')], axis = 1)

# Очистка от простоев и кратких поездок
output = output.loc[(output['dist'] > 0 ) & (output['trip_time'] >= 60)]

[0]


In [7]:
tdf

,lat,lon,t,id,t_diff,lat_prev,lon_prev,coord,coord_prev,dist,velocity,acceleration,movement_flg,velocity_w,acceleration_w,trip_id
0,-6.257188,106.946373,2022-03-15 12:24:58,0,NaN,NaN,NaN,"(-6.25718777778, 106.94637333333333)","(nan, nan)",0.000000,0.000000,0.000000,0,NaN,NaN,NaN
1,-6.257284,106.944613,2022-03-15 12:25:08,0,10.0,-6.257188,106.946373,"(-6.25728388888, 106.94461333333334)","(-6.25718777778, 106.94637333333333)",194.830785,19.483079,1.948308,1,194.830785,19.483079,1.0
2,-6.257325,106.943147,2022-03-15 12:25:18,0,10.0,-6.257284,106.944613,"(-6.257325, 106.94314666666666)","(-6.25728388888, 106.94461333333334)",162.178964,16.217896,0.326518,1,162.178964,3.265182,1.0
3,-6.257380,106.942151,2022-03-15 12:25:30,0,12.0,-6.257325,106.943147,"(-6.25738, 106.94215111116668)","(-6.257325, 106.94314666666666)",110.211173,9.184264,0.586136,1,110.211173,7.033632,1.0
4,-6.257436,106.941049,2022-03-15 12:25:38,0,8.0,-6.257380,106.942151,"(-6.25743555555, 106.94104888883334)","(-6.25738, 106.94215111116668)",121.988025,15.248503,0.758030,1,121.988025,6.064239,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-6.881787,109.410693,2022-03-18 07:05:09,0,10.0,-6.881626,109.410142,"(-6.88178722222, 109.41069333333331)","(-6.88162611112, 109.41014222216668)",63.422130,6.342213,0.051773,1,63.422130,0.517733,29.0
4996,-6.881926,109.411147,2022-03-18 07:05:19,0,10.0,-6.881787,109.410693,"(-6.88192555555, 109.41114666666668)","(-6.88178722222, 109.41069333333331)",52.355836,5.235584,0.110663,1,52.355836,1.106629,29.0
4997,-6.881983,109.411458,2022-03-18 07:05:29,0,10.0,-6.881926,109.411147,"(-6.88198333333, 109.41145777783332)","(-6.88192555555, 109.41114666666668)",34.940519,3.494052,0.174153,1,34.940519,1.741532,29.0
4998,-6.881997,109.411609,2022-03-18 07:05:49,0,20.0,-6.881983,109.411458,"(-6.88199666667, 109.41160888883331)","(-6.88198333333, 109.41145777783332)",16.747491,0.837375,0.132834,1,16.747491,2.656677,29.0


In [8]:
tdf['lat']

0      -6.257188
1      -6.257284
2      -6.257325
3      -6.257380
4      -6.257436
          ...   
4995   -6.881787
4996   -6.881926
4997   -6.881983
4998   -6.881997
4999   -6.881997
Name: lat, Length: 5000, dtype: float64

In [9]:
distance(tdf['lat'], tdf['lon'], tdf['lat_prev'], tdf['lon_prev'])

0            NaN
1       0.194892
2       0.162230
3       0.110246
4       0.122026
          ...   
4995    0.063442
4996    0.052372
4997    0.034951
4998    0.016753
4999    0.000000
Length: 5000, dtype: float64

In [15]:
def distance(s_lat, s_lng, e_lat, e_lng):

   # approximate radius of earth in km
    R = 6371

    s_lat = s_lat*np.pi/180.0                      
    s_lng = np.deg2rad(s_lng)     
    e_lat = np.deg2rad(e_lat)                       
    e_lng = np.deg2rad(e_lng)  

    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2

    return 2 * R * np.arcsin(np.sqrt(d))

In [ ]:
import pandas as pd
import haversine as hs
import warnings
from datetime import datetime
import numpy as np
warnings.filterwarnings('ignore')

# Функция для подготовки поездок конкретного объекта
def data_transforming(tdf):    
    # Сортировка необходима, если мы не знаем, что файл гаранитрованно отсортирован
    # В нашем случае файл уже отсортирован, но оставим сортировку для общего случая
    tdf['t'] = tdf['t'].astype('datetime64')
    tdf.sort_values(by = 't', inplace = True)
    
    # Дельта времени между записями
    tdf['t_diff'] = tdf['t'].diff(1).dt.seconds

    # Вычисление перемещений 
    tdf['lat_prev'] = tdf['lat'].shift(1)
    tdf['lon_prev'] = tdf['lon'].shift(1)
    tdf['coord'] = tdf.apply(lambda x: tuple(x[['lat','lon']]),axis = 1)
    tdf['coord_prev'] = tdf.apply(lambda x: tuple(x[['lat_prev','lon_prev']]),axis = 1)
    tdf['dist'] = tdf[['coord', 'coord_prev']].apply(lambda x: hs.haversine(x['coord'], x['coord_prev']), axis = 1).fillna(0) * 1000
    
    # Вычисление кусочных скоростей и ускорений 
    tdf['velocity'] = (tdf['dist']/tdf['t_diff']).apply(lambda x: 0 if  np.isinf(x) else x).fillna(0)
    tdf['acceleration'] = np.abs((tdf['velocity'].diff(1)/tdf['t_diff']).apply(lambda x: 0 if  np.isinf(x) else x)).fillna(0)
и
    # Флаг признака движения - поможет сгруппировать кусочки поездки в общую поездку
    tdf['movement_flg'] = tdf['dist'].apply(lambda x: 0 if x == 0 else 1)
    
    # Взвешенные скорости и ускорения. Необходимы для вычисления средних ускорений и скоростей.
    tdf['velocity_w'] = tdf['velocity'] * tdf['t_diff']
    tdf['acceleration_w'] = tdf['acceleration'] * tdf['t_diff']
    print(tdf['id'].unique())
    # Пробег в цикле и разметка уникальных поездок. 
    # Необходима для дальнейшей агрегации и вычисления расстояний, скоростей и ускорений 
    j = 0 
    for i in tdf.index[1:]:
        if tdf.loc[i, 'movement_flg'] == tdf.loc[i - 1, 'movement_flg']:
            tdf.loc[i, 'trip_id'] = j
        else:
            j += 1
            tdf.loc[i, 'trip_id'] = j
    
    # Формирование финального результата 
    
    output = pd.concat([tdf.groupby(['trip_id'])['dist'].sum(),
                        tdf.groupby(['trip_id'])['t_diff'].sum().rename('trip_time'),
                       (tdf.groupby('trip_id')['velocity_w'].sum()/ tdf.groupby('trip_id')['t_diff'].sum()).rename('velocity_mean'),
                       (tdf.groupby('trip_id')['acceleration_w'].sum()/ tdf.groupby('trip_id')['t_diff'].sum()).rename('acceleration_mean')], axis = 1)

    # Очистка от простоев и кратких поездок
    output = output.loc[(output['dist'] > 0 ) & (output['trip_time'] >= 60)]
    return output

In [ ]:
sdf

In [ ]:
%%time 
sdf = data_transforming(df.loc[df['id'] == 4])

In [ ]:
import multiprocessing

In [ ]:
multiprocessing.cpu_count()

In [ ]:
import os

In [ ]:
len(os.sched_getaffinity(0))

In [ ]:
sdf

In [ ]:
print(sdf['id'].unique())

#### Дополнительные комментарии: 
1. Есть недостаток у выходного результата: если между двумя записями проходило много секунд, но пройденное расстояние мало, то получается участок с крайне малой скоростью и малым ускорением. На самом деле, объект мог стоять 10 минут и только 20 секунд двигаться, а при расчете параметров время будет взято как 10 минут и 20 сек (а не 20, как нужно).
2. Скорректировать п.1 можно набором правил (например, выкидывать такие участки из рассмотрения) или изменив логику расчета (отказаться от идеи последовательно рассматривать каждую запись а как-то хитро схлопнуть записи по минутам, отбросив секунды)
3. Были попытки отказаться от последовательного цикла по всем записям, но не удалось. Всё же считаю необходимым разметить поездки по признаку движения/покоя
4. Раз уж не удалось отказаться от прохода в цикле по всем записям, то можно было бы производить расчет скоростей и проч. внутри цикла. Я попробовал такой подход, однако он оказался медленнее, чем с использованием предварительного векторизированного расчета встроенного в pandas.
5. Можно поискать дополнительный потенциал ускорения из разряда замены apply на numpy.vectorize и т д

In [ ]:
tdf

In [ ]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))